In [1]:
from catboost import Pool, CatBoostRegressor
from category_encoders import OrdinalEncoder, OneHotEncoder

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, f1_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline

In [2]:
df = pd.read_csv('data/flam_dynamics.csv', decimal=',')
list_of_df = []
for i in range(len(df)):
    new_df = df.iloc[i,:].to_frame(name="flammability").reset_index()
    new_df['region'] = new_df.iloc[1,1]
    new_df['OSM_ID'] = new_df.iloc[0,1]
    new_df = new_df.iloc[3:,:]
    list_of_df.append(new_df)
    
df_flam = pd.concat(list_of_df).reset_index()
df_flam = df_flam.rename(columns={"index": "year"})
df_flam = df_flam.drop(['level_0'], axis=1)

In [3]:
def parse_csv(path, col_name):
    df = pd.read_csv(path, decimal=',')
    list_of_df = []
    for i in range(len(df)):
        new_df = df.iloc[i,:].to_frame(name=col_name).reset_index()
        new_df['OSM_ID'] = new_df.iloc[0,1]
        new_df = new_df.iloc[1:,:]
        list_of_df.append(new_df)

    df = pd.concat(list_of_df).reset_index()
    df = df.rename(columns={"index": "year"})
    df = df.drop(['level_0'], axis=1)
    return df

list_of_df = [parse_csv('data/NDVI_dynamics.csv', 'NDVI'),
                 parse_csv('data/precip_dynamics.csv', 'precipitation'),
                parse_csv('data/stock_dynamics.csv','livestock'),
                 parse_csv('data/t_max_dynamics.csv','t_max')]
list_of_df[0]

,year,NDVI,OSM_ID
0,2001,1858.716392,-3397460.0
1,2002,2216.000243,-3397460.0
2,2003,2211.196412,-3397460.0
3,2004,1992.287442,-3397460.0
4,2005,1989.515517,-3397460.0
...,...,...,...
7261,2017,3626.236649,-214669.0
7262,2018,2851.072608,-214669.0
7263,2019,3592.916096,-214669.0
7264,2020,2831.117611,-214669.0


In [4]:
df_region = pd.read_csv('data/Municipals.csv', encoding="windows-1251", delimiter=';', decimal=',')
df_region = df_region.drop(['Region','NAME', 'Area_ha'], axis=1)

In [5]:
import functools as ft
df = ft.reduce(lambda left, right: pd.merge(left, right, on=['OSM_ID','year']), list_of_df)

In [6]:
df = df.merge(df_flam, on=['OSM_ID','year'])
df = df.merge(df_region, on=['OSM_ID'])
df.head()

,year,NDVI,OSM_ID,precipitation,livestock,t_max,flammability,region,N,E
0,2001,5326.045136,-1957535.0,192.994320,NaN,24.220172,0.0,Камызякский район,45.901151,48.185943
1,2002,5570.302243,-1957535.0,271.498055,NaN,23.694424,0.0,Камызякский район,45.901151,48.185943
2,2003,5105.104907,-1957535.0,239.756170,NaN,22.319267,0.0,Камызякский район,45.901151,48.185943
3,2004,5431.906568,-1957535.0,246.618147,NaN,24.299223,0.0,Камызякский район,45.901151,48.185943
4,2005,5443.259363,-1957535.0,262.731325,NaN,24.166876,0.0,Камызякский район,45.901151,48.185943


In [7]:
df = df.fillna(-9999)

In [8]:
df['flammability'] = np.log1p(df['flammability'])

df['previous_precipitation'] = df['precipitation'].shift(+1)
df.loc[df['year'] == '2001', 'previous_precipitation'] = np.nan
df['previous_precipitation'] = df.groupby(['region']).previous_precipitation.transform(lambda x: x.fillna(x.mean()))
df['previous_NDVI'] = df['precipitation'].shift(+2)
df.loc[df['year'] == '2001', 'previous_NDVI'] = np.nan
df['previous_NDVI'] = df.groupby(['region']).previous_NDVI.transform(lambda x: x.fillna(x.mean()))

In [9]:
df = df.drop(['OSM_ID', 'year'], axis=1)
df.corr()

,NDVI,precipitation,livestock,t_max,flammability,N,E,previous_precipitation,previous_NDVI
NDVI,1.000000,0.524621,-0.167132,-0.360951,-0.126839,0.151653,-0.350645,0.386499,0.299790
precipitation,0.524621,1.000000,-0.230533,-0.741837,-0.016776,0.531376,-0.413004,0.628083,0.574639
livestock,-0.167132,-0.230533,1.000000,0.257455,-0.071312,0.048927,0.254161,-0.231002,-0.161356
t_max,-0.360951,-0.741837,0.257455,1.000000,-0.160536,-0.723279,0.176638,-0.675664,-0.634663
flammability,-0.126839,-0.016776,-0.071312,-0.160536,1.000000,0.273249,0.231180,0.164865,0.026523
N,0.151653,0.531376,0.048927,-0.723279,0.273249,1.000000,0.180268,0.539406,0.532401
E,-0.350645,-0.413004,0.254161,0.176638,0.231180,0.180268,1.000000,-0.431731,-0.405218
previous_precipitation,0.386499,0.628083,-0.231002,-0.675664,0.164865,0.539406,-0.431731,1.000000,0.604253
previous_NDVI,0.299790,0.574639,-0.161356,-0.634663,0.026523,0.532401,-0.405218,0.604253,1.000000


In [10]:
df = df.drop(['region','precipitation'], axis = 1)

In [11]:
from sklearn.compose import ColumnTransformer

col_names = ['NDVI', 'livestock', 't_max', 'previous_precipitation','N','E', 'previous_NDVI']
X = df[col_names]
cv = 10
scalers = (RobustScaler(), StandardScaler(), MinMaxScaler())

regressors = (
    RidgeCV(cv=cv, alphas=[0.0001, 0.001,0.01, 0.1, 1]), 
    LassoCV(cv=cv, alphas=[0.0001, 0.001,0.01, 0.1, 1], n_alphas=[100, 200, 300, 400]), 
    ElasticNetCV(cv=cv, alphas=[0.0001, 0.001,0.01, 0.1, 1, 10], n_alphas=[100, 200, 300, 400])
)

for scaler in scalers:
    for regressor in regressors:
        ct = ColumnTransformer([('scaler', scaler, ['NDVI','livestock', 't_max', 
                                                    'previous_precipitation', 'previous_NDVI'])], remainder='passthrough')
        X_scaled = ct.fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, df['flammability'], test_size=0.20, random_state=42)               
        
        regressor.fit(X_train, y_train)
        y_pred = regressor.predict(X_test)
        
        print('R2: ' + str(regressor.score(X_test, y_test)))
        print('RMSE: ' + str(mean_squared_error(y_test, y_pred, squared=False)))
        print('MAE: ' + str(mean_absolute_error(y_test, y_pred)))
        print('Scaler: ' + scaler.__class__.__name__)
        print('Regressor\'s type:' + regressor.__class__.__name__)
        print('Best Alpha: ' + str(regressor.alpha_))
        print()
#         for col, coef in zip(df.columns, regressor.coef_):
#             print(col, coef)
        print('==' * 20)

R2: 0.11198566188992742
RMSE: 0.9373852300574924
MAE: 0.7598265916993794
Scaler: RobustScaler
Regressor's type:RidgeCV
Best Alpha: 1.0

R2: 0.11196934756737675
RMSE: 0.937393840692537
MAE: 0.7597787556158685
Scaler: RobustScaler
Regressor's type:LassoCV
Best Alpha: 0.0001

R2: 0.11230469070096316
RMSE: 0.9372168320109071
MAE: 0.7597424029076094
Scaler: RobustScaler
Regressor's type:ElasticNetCV
Best Alpha: 0.001

R2: 0.11194948284069661
RMSE: 0.9374043251100801
MAE: 0.7598153133988608
Scaler: StandardScaler
Regressor's type:RidgeCV
Best Alpha: 1.0

R2: 0.1119508402380004
RMSE: 0.9374036086921708
MAE: 0.7597868775284422
Scaler: StandardScaler
Regressor's type:LassoCV
Best Alpha: 0.0001

R2: 0.1121984342302953
RMSE: 0.9372729224125809
MAE: 0.7597778171129771
Scaler: StandardScaler
Regressor's type:ElasticNetCV
Best Alpha: 0.001

R2: 0.11199453630718548
RMSE: 0.9373805461418789
MAE: 0.7598701031207253
Scaler: MinMaxScaler
Regressor's type:RidgeCV
Best Alpha: 0.1

R2: 0.11212061605807155
R

In [12]:
#Test Default CatBoost
print("Testing default Catboost")

scalers = (RobustScaler(), StandardScaler(), MinMaxScaler())
for scaler in scalers:
    
    ct = ColumnTransformer([('scaler', scaler, ['NDVI','livestock', 't_max', 
                                                'previous_precipitation', 'previous_NDVI'])], remainder='passthrough')
    
    X_scaled = ct.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, 
                                                        df['flammability'], 
                                                        test_size=0.20, random_state=42)  
    model = CatBoostRegressor(logging_level='Silent')
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    rmse = (np.sqrt(mean_squared_error(y_test, pred)))
    r2 = r2_score(y_test, pred)
    
    print('R2: ' + str(r2))
    print('RMSE: ' + str(mean_squared_error(y_test, y_pred, squared=False)))
    print('MAE: ' + str(mean_absolute_error(y_test, y_pred)))
    print('Scaler: ' + scaler.__class__.__name__)
    #print(model.get_all_params())
    print('==' * 20)

Testing default Catboost
R2: 0.4992239937863856
RMSE: 0.9373488863563302
MAE: 0.7598173567389651
Scaler: RobustScaler
R2: 0.49952372237862785
RMSE: 0.9373488863563302
MAE: 0.7598173567389651
Scaler: StandardScaler
R2: 0.49113087341111716
RMSE: 0.9373488863563302
MAE: 0.7598173567389651
Scaler: MinMaxScaler


In [13]:
scaler = RobustScaler()
ct = ColumnTransformer([('scaler', scaler, ['NDVI','livestock', 't_max', 
                                                'previous_precipitation', 'previous_NDVI'])], remainder='passthrough')
X_scaled = ct.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, 
                                                    df['flammability'], 
                                                    test_size=0.20, random_state=42)
model = CatBoostRegressor(logging_level='Silent')

grid = {
        'learning_rate': [0.01, 0.03, 0.1, 0.5],
        'depth': [4, 6, 8],
        'l2_leaf_reg': [1, 3, 5, 7],
        'iterations': [250, 500, 1000]
       }

grid_search = model.grid_search(grid, X=X_train, y=y_train, plot=True, verbose=False)
#grid.fit(X_train, y_train)

pred = model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
r2 = r2_score(y_test, pred)

print('R2: ' + str(r2))
print('RMSE: ' + str(mean_squared_error(y_test, y_pred, squared=False)))
print('MAE: ' + str(mean_absolute_error(y_test, y_pred)))
print('Scaler: ' + scaler.__class__.__name__)
print('Best params: ' + str(grid_search['params']))
#print(model.get_all_params())
print('==' * 20)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

R2: 0.4837875569048684
RMSE: 0.9373488863563302
MAE: 0.7598173567389651
Scaler: RobustScaler
Best params: {'depth': 8, 'l2_leaf_reg': 7, 'iterations': 250, 'learning_rate': 0.1}


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, 
                                                        df['flammability'], 
                                                        test_size=0.20, random_state=42) 
param_grid = {
    'C': [1, 5, 100, 200, 1000],
    'epsilon': [0.01, 0.1, 0.05, 0.0003, 1, 0.2, 5, 10],
    'gamma': [0.001, 0.1, 1, 5, 10, 100]
}
grid_search = GridSearchCV(estimator = SVR(), param_grid = param_grid, cv = 10, n_jobs = -1, verbose = 2)
regr = make_pipeline(RobustScaler(), grid_search)


regr.fit(X_train, y_train)
print(regr.score(X_test, y_test))
print(regr[1].best_params_)

Fitting 10 folds for each of 240 candidates, totalling 2400 fits
0.4082934094268271
{'C': 1, 'epsilon': 0.2, 'gamma': 1}


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, 
                                                        df['flammability'], 
                                                        test_size=0.20, random_state=43)
knn = KNeighborsRegressor()
k_range = list(range(1, 8))
param_grid = dict(n_neighbors=k_range)
  
# defining parameter range
grid = GridSearchCV(knn, param_grid, cv=10, verbose=1)

# fitting the model for grid search
grid.fit(X_train, y_train)
print(grid.best_estimator_)
grid.score(X_test, y_test)

Fitting 10 folds for each of 7 candidates, totalling 70 fits
KNeighborsRegressor()


0.4894148047928363